## 2. Loading Data and Running Vector Searches
Now that you have created and run a simple Streamlit application that essentially acts as a basic LLM, let's load some data into InterSystems IRIS and run vector searches on this data. We will later connect this vector storage to the Streamlit application so that the chat app can utilize this data.

We will use the *FastEmbed* embeddings model to vectorize our data, and we'll use Langchain to load and interact with the data. Langchain provides several advantages in building a RAG application, including streamlining the retrieval process, adding conversation history, enabling guardrails to keep your application within its intended usage, and more. We will implement these features of the chat later in the workshop.

The following block of code is used to manage environment variables, specifically for loading and setting the OpenAI API key. It begins by importing necessary modules for operating system interactions and secure password input. Run the block below to load these settings.

In [ ]:
import getpass
import os


The next block imports a variety of libraries and modules for completing advanced language processing tasks. These include handling and storing documents, loading textual and JSON data, splitting text based on character count, and utilizing embeddings from OpenAI, Hugging Face, and potentially faster embedding methods. 

In [ ]:
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader, JSONLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.fastembed import FastEmbedEmbeddings

from langchain_iris import IRISVector
from sqlalchemy import create_engine, text


### View the Case Reports data
Before loading this data into InterSystems IRIS, let's take a quick look at it. We can use a Pandas DataFrame to easily format and view the data, which currently exists within a `json lines` file, which is a file format where each line is a complete JSON object, separated by new lines. This format is particularly useful for handling large datasets or streams of data, because it allows for reading, writing, and processing one line (or one JSON object) at a time, rather than needing to load an entire file into memory at once.

Run the snippet below to create a DataFrame and view the first 10 case reports in the data set.

In [ ]:
import pandas as pd

# Load JSONL file into DataFrame
file_path = './data/healthcare/augmented_notes_100.jsonl'
df_cases = pd.read_json(file_path, lines=True)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)
df_cases.head(10)

### Load the case reports into InterSystems IRIS via Langchain
Now you'll load, split, and prepare a set of 100 case reports for embedding.

1. Load the Data — Use a JSONLoader to read the file, specifying json_lines=True to handle the JSON Lines format.

2. Split into Chunks — After loading, split the text into chunks of 2,500 characters with a 100-character overlap. This ensures each case report becomes one chunk while preserving important context across boundaries.

3. Why Chunking Matters — Chunking makes large documents easier to manage and embed. The overlap improves embedding quality by retaining contextual continuity between chunks.

Later in this section, you’ll apply further chunking.

Run the code block below to generate the chunks. You should see 100 chunks as output, since each case report fits within the defined size.

In [ ]:
## load data from json lines file
loader = JSONLoader(
    file_path='./data/healthcare/augmented_notes_100.jsonl',
    jq_schema='.note',
    json_lines=True
)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

docs = text_splitter.split_documents(documents)
print(f"Got {len(docs)} chunks")

With chunks created, let's prepare to load the data into InterSystems IRIS. Run the following block to create and print the connection string that will be used to connect to InterSystems IRIS. 

In [ ]:
username = '_SYSTEM'
password = 'SYS'
hostname = 'IRIS'
port = 1972
namespace = 'IRISAPP'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"
print(CONNECTION_STRING)

Next, let's initialize a database in InterSystems IRIS, which you will populate with the case reports. 

This setup is essential for applications involving search and retrieval of information where the semantic content of the documents is more important than their keyword content. The vector store uses embeddings to perform similarity searches, offering significant advantages over traditional search methods by understanding the context and meaning embedded within the text.

Run the block below to load your data and embeddings into InterSystems IRIS. Note that we are using the *FastEmbed* model to create our embeddings, but your application could use a variety of different embeddings models.

This may take a few moments.

In [ ]:
embeddings = FastEmbedEmbeddings()
# embeddings = OpenAIEmbeddings() # Only use OpenAIEmbeddings here if issues with FastEmbed

COLLECTION_NAME = "case_reports"
db = IRISVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

print("Embeddings generation for case reports complete!")

Confirm that there are 100 documents in your InterSystems IRIS vector storage by running the following block.

In [ ]:
print(f"Number of docs in vector store: {len(db.get()['ids'])}")

### Try out vector search
With the case reports now embedded and stored, you can run a vector search. In the example below, we use the query:

- *"Have any children presented with knee injuries?"*

The code returns matching documents and their similarity scores, which reflect how closely each result matches the query.

**How It Works:**
- The query is converted into an embedding using the same FastEmbed model used for the case reports.
- Langchain's `similarity_search_with_score` function searches the IRIS vector store for the most semantically similar results.

🔎 **Note:** Lower scores = higher similarity.

In [ ]:
query = "Have any patients presented with knee injuries?"
docs_with_score = db.similarity_search_with_score(query)

Run the following block to print the returned documents along with their scores.

In [ ]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

Next, you will enter a search phrase of your choice and perform a similarity search on that phrase.

Set the *query* variable to a word or phrase of your choice; you can reference keywords from some of the case reports you saw in the data set when initially browsing it, if you'd like. Or you can choose any phrase you wish. Then, run the block.

In [ ]:
query="Pulmonary diseases"
docs_with_score = db.similarity_search_with_score(query)

Run the block below to print the most similar results. Observe the similarity scores, keeping in mind that the closer to 0 it is, the closer in similarity the document is to your search phrase.

In [ ]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

### Experiment with chunk sizes
Next, let's try experimenting with varied chunk sizes. Run the block below, which is the same as your previous iteration for loading and chunking, but this time uses a chunk size of 400 instead of 2,500. Observe the change in the number of chunks that were created.

In [ ]:
## load data from json lines file
loader = JSONLoader(
    file_path='./data/healthcare/augmented_notes_100.jsonl',
    jq_schema='.note',
    json_lines=True
)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

docs = text_splitter.split_documents(documents)
print(f"Got {len(docs)} chunks")

Now let's load this newly chunked data into a separate data store within InterSystems IRIS.

Once again, this may take several moments while embeddings are created for each chunk.

In [ ]:
# embeddings = OpenAIEmbeddings()
embeddings = FastEmbedEmbeddings()

COLLECTION_NAME = "case_reports-chunked"
dbchunked = IRISVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

print("Embeddings generation for case reports complete!")

Confirm how many documents are loaded into this second data structure in InterSystems IRIS by running the block below.

In [ ]:
print(f"Number of docs in vector store: {len(dbchunked.get()['ids'])}")

Now let's try another vector search against the more finely-chunked data. Run the module below, optionally replacing your search term with the one you chose earlier. Whichever term you choose, observe how the results and similarity scores may differ slightly from before, when larger chunks were used.

Also, you may notice that separate chunks can be part of the same case report. In practice, you may leverage a separate lookup table that could synthesize chunks alongside their original records. Here, we are simply returning the most relevant chunks.

In [ ]:
query = "Have any children presented with knee injuries?"
docs_with_score = dbchunked.similarity_search_with_score(query)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

### Prepare retrieval mechanism for RAG app
Now that we have experimented with vector searches and chunk sizes, let's set up a retriever for our database. A retriever is an essential component in information retrieval systems, as it allows us to fetch relevant documents based on a query efficiently. By converting the database into a retriever, we enhance our ability to interact with the data, enabling more advanced search and retrieval operations.

The `as_retriever()` method transforms the database into a retriever object. This object can then be used to perform various retrieval tasks, making it a versatile tool for working with our embedded documents.

Run the block below to create the retriever and print it to confirm its setup. We'll use the more finely chunked data, consisting of 764 chunks, for this retriever.

This retriever will be used for the RAG application later in the workshop.

In [ ]:
retriever = dbchunked.as_retriever()
print(retriever)